In [4]:
from fastai.collab import *
from fastai.tabular import *

import matplotlib
import matplotlib.pyplot as plt
import random
import numpy as np

import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, TabularDataset
from torchtext.data import Iterator, BucketIterator

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 7.0)


In [38]:
user,item,title = 'userId','movieId','title'
path = untar_data(URLs.ML_SAMPLE)
path
ratings = pd.read_csv(path/'ratings.csv')
ratings.head()
y_range = [0,5.5]


In [109]:
data = CollabDataBunch.from_df(ratings, seed=42, valid_pct=0.1, item_name = 'movieId')

In [110]:
x,y = data.train_ds[0]
print(x)

userId 73; movieId 1097; 


In [111]:
xb,yb = data.one_batch()

In [112]:
data.show_batch(rows=3)

userId,movieId,target
213,32,1.5
157,344,1.0
299,1732,5.0


In [127]:
learn = collab_learner(data, n_factors=40, y_range=y_range, wd=1e-1)

In [116]:
# learn.lr_find()
# learn.recorder.plot(skip_end=15)

In [117]:
learn.fit_one_cycle(1, 5e-3)

epoch,train_loss,valid_loss,time
0,1.402211,1.157498,00:56


In [96]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(101, 50)
  (i_weight): Embedding(101, 50)
  (u_bias): Embedding(101, 1)
  (i_bias): Embedding(101, 1)
)

In [58]:
movies

,userId,movieId,rating,timestamp
0,73,1097,4.0,1255504951
1,561,924,3.5,1172695223
2,157,260,3.5,1291598691
3,358,1210,5.0,957481884
4,130,316,2.0,1138999234
...,...,...,...,...
6026,384,377,4.0,1153501701
6027,262,587,1.5,1434024940
6028,580,153,2.0,1165899265
6029,585,1,4.0,974607482


In [63]:
np.array([585,1])

array([585,   1])

In [62]:
learn.get_idx(np.array([585,1]))

You're trying to access an item that isn't in the training data.
                  If it was in your original data, it may have been split such that it's only in the validation set now.


In [17]:
path

PosixPath('/Users/spencerbraun/.fastai/data/movie_lens_sample')

In [22]:
! cat /Users/spencerbraun/.fastai/data/movie_lens_sample/ratings.csv | head











cat: stdout: Broken pipe


In [118]:
movies = pd.read_csv(path/'ratings.csv',  delimiter=',', encoding='latin-1')
movies.head()
rating_movie = movies#ratings.merge(movies[[item, title]])
# rating_movie.head()

In [121]:
rating_movie

,userId,movieId,rating,timestamp
0,73,1097,4.0,1255504951
1,561,924,3.5,1172695223
2,157,260,3.5,1291598691
3,358,1210,5.0,957481884
4,130,316,2.0,1138999234
...,...,...,...,...
6026,384,377,4.0,1153501701
6027,262,587,1.5,1434024940
6028,580,153,2.0,1165899265
6029,585,1,4.0,974607482


In [120]:
doc(learn.bias)

In [99]:
top_movies

array([  356,  1270,   296,   260, ...,  4973,   316,    10, 58559])

In [128]:
g = movies.groupby('movieId')['rating'].count()
top_movies = g.sort_values(ascending=False).index.values[:1000]
top_movies[:10]


array([ 356, 1270,  296,  260,  480,  593, 1580, 1265, 1196, 2571])

In [133]:
    
learn.summary
# movie_bias.shape

<bound method model_summary of CollabLearner(data=TabularDataBunch;

Train: LabelList (5428 items)
x: CollabList
userId 73; movieId 1097; ,userId 561; movieId 924; ,userId 157; movieId 260; ,userId 358; movieId 1210; ,userId 130; movieId 316; 
y: FloatList
4.0,3.5,3.5,5.0,2.0
Path: .;

Valid: LabelList (603 items)
x: CollabList
userId 306; movieId 2628; ,userId 605; movieId 3793; ,userId 313; movieId 4886; ,userId 468; movieId 1136; ,userId 380; movieId 539; 
y: FloatList
3.0,2.0,4.5,4.0,1.5
Path: .;

Test: None, model=EmbeddingDotBias(
  (u_weight): Embedding(101, 40)
  (i_weight): Embedding(101, 40)
  (u_bias): Embedding(101, 1)
  (i_bias): Embedding(101, 1)
), opt_func=functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99)), loss_func=FlattenedLoss of MSELoss(), metrics=[], true_wd=True, bn_wd=True, wd=0.1, train_bn=True, path=PosixPath('.'), model_dir='models', callback_fns=[functools.partial(<class 'fastai.basic_train.Recorder'>, add_time=True, silent=False)], callba

In [100]:
mean_ratings

movieId
1        3.760274
10       3.285714
32       3.855932
34       3.443396
39       3.406250
           ...   
6377     3.740000
6539     3.907407
7153     4.034483
8961     3.944444
58559    4.062500
Name: rating, Length: 100, dtype: float64

In [53]:
mean_ratings = rating_movie.groupby('movieId')['rating'].mean()
movie_ratings = [(i, mean_ratings.loc[i]) for i in top_movies]

In [101]:
movie_ratings

[(356, 3.8850574712643677),
 (1270, 3.9879518072289155),
 (296, 4.475609756097561),
 (260, 4.25625),
 (480, 3.537974683544304),
 (593, 4.177215189873418),
 (1580, 3.4358974358974357),
 (1265, 3.7467532467532467),
 (1196, 4.2727272727272725),
 (2571, 4.1558441558441555),
 (2858, 4.1866666666666665),
 (1198, 4.20945945945946),
 (318, 4.331081081081081),
 (1, 3.76027397260274),
 (2762, 3.9794520547945207),
 (780, 3.2708333333333335),
 (1210, 4.176056338028169),
 (377, 3.3840579710144927),
 (4993, 4.125),
 (457, 3.860294117647059),
 (47, 4.125),
 (2628, 3.0522388059701493),
 (608, 4.1716417910447765),
 (1097, 3.873134328358209),
 (1721, 3.1343283582089554),
 (1240, 3.8507462686567164),
 (2716, 3.7954545454545454),
 (589, 3.962121212121212),
 (110, 3.8257575757575757),
 (858, 4.484848484848484),
 (2028, 4.0606060606060606),
 (1291, 3.8615384615384616),
 (5952, 3.9615384615384617),
 (2959, 4.276923076923077),
 (2918, 3.9307692307692306),
 (588, 3.640625),
 (527, 4.2890625),
 (1923, 3.484375)

In [ ]:
item0 = lambda o:o[0]

In [ ]:
sorted(movie_ratings, key=item0)[:15]

In [ ]:
sorted(movie_ratings, key=lambda o: o[0], reverse=True)[:15]

In [ ]:
movie_w = learn.weight(top_movies, is_item=True)
movie_w.shape

In [ ]:
movie_pca = movie_w.pca(3)
movie_pca.shape

In [ ]:
fac0,fac1,fac2 = movie_pca.t()
movie_comp = [(f, i) for f,i in zip(fac0, top_movies)]

In [ ]:
idxs = np.random.choice(len(top_movies), 50, replace=False)
idxs = list(range(50))
X = fac0[idxs]
Y = fac2[idxs]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(top_movies[idxs], X, Y):
    plt.text(x,y,i, color=np.random.rand(3)*0.7, fontsize=11)
plt.show()